In [1]:
!pip install --upgrade "ibm-watson>=4.0.1"

     |████████████████████████████████| 327kB 6.1MB/s eta 0:00:01
     |████████████████████████████████| 204kB 23.3MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/cc/33/18/90271d1fb07b27d9958bd89d5e4858196c844b69b089050cdc
  Stored in directory: /home/dsxuser/.cache/pip/wheels/12/e5/b5/90c69cc81edf5b95cbec373c83c3530266fdae8266037380df
Successfully built ibm-watson ibm-cloud-sdk-core


In [2]:
# @hidden_cell
import json
from ibm_watson import NaturalLanguageClassifierV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('akM7V1ISmAWD_JbOWeBFgfOvgCtBX6H6g9FfM4dTf4Vb',url='https://iam.cloud.ibm.com/identity/token')
nlc = NaturalLanguageClassifierV1(authenticator)
nlc.set_default_headers({'url':'https://gateway.watsonplatform.net/natural-language-classifier/api'})

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_d583ed9871cd4c0f989be20d8872167e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='UWesfz1VTTZeniaqrn7yuKnXLHifp180n9DtgSaLgowe',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d583ed9871cd4c0f989be20d8872167e.get_object(Bucket='laboratriowatsonstudio-donotdelete-pr-pbreu1ixybb27s',Key='test_cerveja.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

test = pd.read_csv(body, header=None)
test.head()


,0,1
0,Trigo é o vilão dos alimentos! Elimine cerveja...,saude
1,"Com a redução de viagens, e de emissão de carb...",ambiente
2,"Cerveja com arte, sustentabilidade, re-uso ou ...",ambiente
3,"Cerveja evita engordar, aponta pesquisa <a tar...",saude
4,"Depois de uma corrida de kart, uma alcatra na ...",comportamento


In [4]:
predicted = []
for i in range(0,test.shape[0]): 
    classes = nlc.classify('94904ex626-nlc-66',test.iloc[i,0]).get_result()
    predicted.append(classes['top_class'])

In [5]:
resultados = pd.DataFrame({'real': test.iloc[:,1], 'predicted': predicted})

In [6]:
resultados.head()

,real,predicted
0,saude,saude
1,ambiente,ambiente
2,ambiente,ambiente
3,saude,saude
4,comportamento,comportamento


In [7]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
report = classification_report(test.iloc[:,1], predicted)

In [8]:
print(report)

               precision    recall  f1-score   support

     ambiente       1.00      0.96      0.98        23
comportamento       0.81      0.81      0.81        21
 conhecimento       0.76      0.84      0.80        19
     economia       1.00      0.95      0.98        22
  responsavel       1.00      0.95      0.97        19
        saude       0.87      0.91      0.89        22

    micro avg       0.90      0.90      0.90       126
    macro avg       0.91      0.90      0.90       126
 weighted avg       0.91      0.90      0.91       126



In [9]:
confusion_matrix(test.iloc[:,1], predicted)

array([[22,  0,  1,  0,  0,  0],
       [ 0, 17,  2,  0,  0,  2],
       [ 0,  3, 16,  0,  0,  0],
       [ 0,  0,  1, 21,  0,  0],
       [ 0,  0,  0,  0, 18,  1],
       [ 0,  1,  1,  0,  0, 20]])